In [216]:
import os
import cv2
import numpy as np
import glob as glob
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [217]:
path = "image/base"
resize_path = 'image/resize'
folders = os.listdir(path)

In [218]:
classes = [f for f in folders if os.path.isdir(os.path.join(path, f))]
n_classes = len(classes)
n_classes

3

In [233]:
X = []
Y = []

In [234]:
for label, class_name in enumerate(classes):
    files = glob.glob(path + "/" + class_name + "/*.jpg")
    for file in files:
        img = cv2.imread(file)
        print(os.path.basename(file))
        img = cv2.resize(img, dsize=(224, 224))
        X.append(img)
        Y.append(label)

front (1).jpg
front (10).jpg
front (11).jpg
front (12).jpg
front (13).jpg
front (14).jpg
front (15).jpg
front (16).jpg
front (17).jpg
front (18).jpg
front (19).jpg
front (2).jpg
front (20).jpg
front (3).jpg
front (4).jpg
front (5).jpg
front (6).jpg
front (7).jpg
front (8).jpg
front (9).jpg
restaurant (1).jpg
restaurant (10).jpg
restaurant (11).jpg
restaurant (12).jpg
restaurant (13).jpg
restaurant (14).jpg
restaurant (15).jpg
restaurant (16).jpg
restaurant (17).jpg
restaurant (18).jpg
restaurant (19).jpg
restaurant (2).jpg
restaurant (20).jpg
restaurant (3).jpg
restaurant (4).jpg
restaurant (5).jpg
restaurant (6).jpg
restaurant (7).jpg
restaurant (8).jpg
restaurant (9).jpg
toilet (1).jpg
toilet (10).jpg
toilet (11).jpg
toilet (12).jpg
toilet (13).jpg
toilet (14).jpg
toilet (15).jpg
toilet (16).jpg
toilet (17).jpg
toilet (18).jpg
toilet (19).jpg
toilet (2).jpg
toilet (20).jpg
toilet (3).jpg
toilet (4).jpg
toilet (5).jpg
toilet (6).jpg
toilet (7).jpg
toilet (8).jpg
toilet (9).jpg


In [235]:
X = np.array(X)
X = X.astype('float32')
X /= 255.0

In [236]:
Y = np.array(Y)
Y = to_categorical(Y, n_classes)
print(Y)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [237]:
X_train = X
Y_train = Y
print(X_train.shape)
print(Y_train.shape)

(60, 224, 224, 3)
(60, 3)


In [238]:
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.models import model_from_json
from keras.models import Model
from keras.layers import Input, Activation, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

In [239]:
input_tensor = Input(shape=(224,224,3))
base_model = VGG16(weights='imagenet', input_tensor=input_tensor, include_top=False)

top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(n_classes, activation='softmax'))

model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

for layer in model.layers[:15]:
    layer.trainable = False
    
print('# layers=', len(model.layers))

model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

model.summary()

# layers= 20
Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 1

In [240]:
model.fit(X_train, Y_train, epochs=20, batch_size=16)

score = model.evaluate(X_train, Y_train, batch_size=16)

Epoch 1/20
4/4 [==============================] - 12s 3s/step - loss: 1.2500 - accuracy: 0.1500
Epoch 2/20
4/4 [==============================] - 10s 2s/step - loss: 1.1913 - accuracy: 0.2000
Epoch 3/20
4/4 [==============================] - 9s 2s/step - loss: 1.1385 - accuracy: 0.2333
Epoch 4/20
4/4 [==============================] - 10s 2s/step - loss: 1.0848 - accuracy: 0.3167
Epoch 5/20
4/4 [==============================] - 10s 2s/step - loss: 1.0357 - accuracy: 0.4167
Epoch 6/20
4/4 [==============================] - 9s 2s/step - loss: 0.9906 - accuracy: 0.4667
Epoch 7/20
4/4 [==============================] - 9s 2s/step - loss: 0.9462 - accuracy: 0.6333
Epoch 8/20
4/4 [==============================] - 9s 2s/step - loss: 0.9038 - accuracy: 0.7167
Epoch 9/20
4/4 [==============================] - 9s 2s/step - loss: 0.8635 - accuracy: 0.7667
Epoch 10/20
4/4 [==============================] - 10s 2s/step - loss: 0.8258 - accuracy: 0.8500
Epoch 11/20
4/4 [===========================

In [241]:
import pickle
pickle.dump(classes, open('backend/models/classes.sav', 'wb'))
model.save('backend/models/cnn.h5')

In [242]:
from keras.models import load_model
import pickle
import cv2
import glob

In [243]:
model = load_model('backend/models/cnn.h5')
classes = pickle.load(open('backend/models/classes.sav', 'rb'))

In [244]:
img = cv2.imread('./image/base/front/front (2).jpg')
img = cv2.resize(img, dsize=(224, 224))
img = img.astype('float32')
img /= 255.0
img = img[None, ...]
result = model.predict(img)
pred = result.argmax()

img = cv2.imread('./image/base/front/front (2).jpg')
cv2.putText(img, classes[pred] + ' ' + str(result[0][pred] * 100) + '%', (0, 50), cv2.FONT_HERSHEY_PLAIN, 4, (0, 0, 0), 5, cv2.LINE_AA)
cv2.imwrite('./image/output/' + str(classes[pred]) + '/front (2).jpg', img)

1/1 [==============================] - 0s 339ms/step


True

In [245]:
files = glob.glob('./image/all/*')
for file in files:
    img = cv2.imread(file)
    img = cv2.resize(img, dsize=(224, 224))
    img = img.astype('float32')
    img /= 255.0
    img = img[None, ...]
    result = model.predict(img)
    pred = result.argmax()
    img = cv2.imread(file)
    cv2.putText(img, classes[pred] + ' ' + str(result[0][pred] * 100) + '%', (0, 50), cv2.FONT_HERSHEY_PLAIN, 4, (0, 0, 0), 5, cv2.LINE_AA)
    cv2.putText(img, classes[result.argmin()] + ' ' + str(result[0][result.argmin()] * 100) + '%', (0, 150), cv2.FONT_HERSHEY_PLAIN, 4, (0, 0, 0), 5, cv2.LINE_AA)
    cv2.imwrite('./image/output/' + str(classes[pred]) + '/' + os.path.basename(file), img)

1/1 [==============================] - 0s 138ms/step
